In [ ]:
import pandas as pd
from qlib.data import D
import qlib
from modeltester import AlphaEval
qlib.init(
    provider_uri="path/to/your/qlib_data",
    region="cn"
)

In [ ]:
eval = AlphaEval(
    factor_expressions=[
        "Slope($change, 5)",
        "Resi($change, 64)",
        "IdxMin($open, 5)",
        "Add(IdxMin($open, 5), Std($vwap, 5))"       
    ], 
    weights=[0.31648424, 0.49929312, 0.08017774, 0.1040449],
    test_start_date="2021-01-01",
    test_end_date="2024-12-31",
    instruments = D.instruments(market='all'),
    daily_normalize=True
)

In [ ]:
eval.run()
eval.summary()